In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Allergies"
cohort = "GSE185658"

# Input paths
in_trait_dir = "../../input/GEO/Allergies"
in_cohort_dir = "../../input/GEO/Allergies/GSE185658"

# Output paths
out_data_file = "../../output/preprocess/Allergies/GSE185658.csv"
out_gene_data_file = "../../output/preprocess/Allergies/gene_data/GSE185658.csv"
out_clinical_data_file = "../../output/preprocess/Allergies/clinical_data/GSE185658.csv"
json_path = "../../output/preprocess/Allergies/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19"
!Series_summary	"Balanced immune responses in airways of patients with asthma are crucial to succesful clearance of viral infection and proper asthma control."
!Series_summary	"We used microarrays to detail the global programme of gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo."
!Series_overall_design	"Bronchial brushings from control individuals and patients with asthma around two weeks before (day -14) and four days after (day 4) experimental in vivo rhinovirus infection were used for RNA isolation and hybrydyzation with Affymetric microarrays."
Sample Characteristics Dictionary:
{0: ['time: DAY14', 'time: DAY4'], 1: ['group: AsthmaHDM', 'group: AsthmaHDMNeg', 'group: Healthy'], 2: ['donor: DJ144', 'donor: DJ113', 'donor: DJ139', 'donor

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Dict, Any, Optional, Callable

# 1. Evaluate Gene Expression Data Availability
# Based on the background information, this dataset contains microarray gene expression data
# from bronchial brushings, which is suitable for our study.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 & 2.2 Trait (Allergies/Asthma)
# Looking at row 1, we have information about asthma groups
trait_row = 1

def convert_trait(value):
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Asthma is a form of allergic disease, so we can use this as our trait
    if 'Asthma' in value:
        return 1  # Has asthma/allergies
    elif 'Healthy' in value:
        return 0  # No asthma/allergies
    return None

# Age: Not available in the sample characteristics
age_row = None

def convert_age(value):
    return None

# Gender: Not available in the sample characteristics 
gender_row = None

def convert_gender(value):
    return None

# 3. Save Metadata - Initial Filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data is available)
if trait_row is not None:
    # Load the clinical data file
    try:
        # First, look for a file that may contain the sample characteristics
        files = os.listdir(in_cohort_dir)
        clinical_files = [f for f in files if f.endswith('_series_matrix.txt') or f.endswith('_family.soft')]
        
        if clinical_files:
            # Read sample characteristics from the matrix file
            clinical_file_path = os.path.join(in_cohort_dir, clinical_files[0])
            with open(clinical_file_path, 'r') as f:
                lines = f.readlines()
            
            # Parse the characteristics from the file
            characteristics_dict = {}
            for line in lines:
                if line.startswith('!Sample_characteristics_ch1'):
                    parts = line.strip().split('\t')
                    if len(parts) > 1:
                        characteristic = parts[1].strip('"')
                        idx = None
                        for i, prefix in enumerate(['time:', 'group:', 'donor:']):
                            if characteristic.startswith(prefix):
                                idx = i
                                break
                        
                        if idx is not None:
                            if idx not in characteristics_dict:
                                characteristics_dict[idx] = []
                            characteristics_dict[idx].append(characteristic)
            
            # Create a DataFrame from the characteristics dictionary
            sample_ids = [f'Sample_{i+1}' for i in range(len(lines))]
            clinical_data = pd.DataFrame(index=sample_ids)
            
            for idx, characteristics in characteristics_dict.items():
                if len(characteristics) > 0:
                    # Ensure the length matches the number of samples
                    while len(characteristics) < len(sample_ids):
                        characteristics.append(None)
                    
                    # Truncate if there are more characteristics than samples
                    characteristics = characteristics[:len(sample_ids)]
                    
                    # Add to DataFrame
                    clinical_data[f'Characteristic_{idx}'] = characteristics
            
            # Extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the extracted clinical data
            preview = preview_df(selected_clinical_df)
            print("Preview of selected clinical features:")
            print(preview)
            
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            
            # Save the clinical data
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print(f"No series matrix or family soft file found in {in_cohort_dir}")
            # Since we validated trait is available but cannot extract it,
            # we should update the is_trait_available flag
            is_trait_available = False
            validate_and_save_cohort_info(
                is_final=False,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available
            )
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        # Update metadata since we encountered an error
        is_trait_available = False
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )


No series matrix or family soft file found in ../../input/GEO/Allergies/GSE185658


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be probe IDs rather than standard human gene symbols
# Standard human gene symbols would typically be alphanumeric strings like "BRCA1", "TP53", etc.
# The numeric format (7892501, 7892502, etc.) indicates these are likely probe IDs from a microarray platform
# These would need to be mapped to actual gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // ---

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain the probe IDs and gene symbols
# From examining the preview, we can see that 'ID' column in gene_annotation contains probe IDs
# and 'gene_assignment' contains gene symbol information

# 2. Create a mapping dataframe with probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the first few rows of the gene expression data after mapping
print("First 10 rows of gene expression data after mapping:")
print(gene_data.head(10))


First 10 rows of gene expression data after mapping:
      GSM5621296  GSM5621297  GSM5621298  GSM5621299  GSM5621300  GSM5621301  \
Gene                                                                           
A-     33.880148   32.989969   33.356958   33.529188   33.455785   32.280051   
A-3-    0.851915    0.839322    0.822292    0.933672    0.865422    0.868687   
A-52    1.394083    1.422103    1.444974    1.397532    1.442924    1.344287   
A-E     1.440662    1.478237    1.454249    1.470040    1.423566    1.438162   
A-I     2.065450    1.999045    2.063801    2.015143    2.068438    1.889955   
A-II    1.004147    0.946524    0.979957    0.994018    0.971314    1.031967   
A-IV    2.134692    2.130700    2.140920    2.180278    2.067903    2.147950   
A-V     0.611307    0.598379    0.582965    0.595110    0.557132    0.572635   
A0      0.658540    0.665759    0.683064    0.651565    0.675683    0.673642   
A1     87.880285   89.162301   88.969654   88.346438   89.420356   

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo, as described in the study 'Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19'."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (24221, 48)


Normalized gene data saved to ../../output/preprocess/Allergies/gene_data/GSE185658.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM5621296': [1.0], 'GSM5621297': [1.0], 'GSM5621298': [1.0], 'GSM5621299': [1.0], 'GSM5621300': [1.0], 'GSM5621301': [0.0], 'GSM5621302': [0.0], 'GSM5621303': [1.0], 'GSM5621304': [1.0], 'GSM5621305': [1.0], 'GSM5621306': [1.0], 'GSM5621307': [0.0], 'GSM5621308': [1.0], 'GSM5621309': [1.0], 'GSM5621310': [1.0], 'GSM5621311': [0.0], 'GSM5621312': [1.0], 'GSM5621313': [1.0], 'GSM5621314': [0.0], 'GSM5621315': [1.0], 'GSM5621316': [0.0], 'GSM5621317': [1.0], 'GSM5621318': [1.0], 'GSM5621319': [0.0], 'GSM5621320': [1.0], 'GSM5621321': [0.0], 'GSM5621322': [0.0], 'GSM5621323': [1.0], 'GSM5621324': [1.0], 'GSM5621325': [1.0], 'GSM5621326': [1.0], 'GSM5621327': [1.0], 'GSM5621328': [0.0], 'GSM5621329': [1.0], 'GSM5621330': [1.0], 'GSM5621331': [1.0], 'GSM5621332': [1.0], 'GSM5621333': [1.0], 'GSM5621334': [0.0],

Linked data shape after handling missing values: (48, 24222)
Checking for bias in trait distribution...
For the feature 'Allergies', the least common label is '0.0' with 14 occurrences. This represents 29.17% of the dataset.
The distribution of the feature 'Allergies' in this dataset is fine.

Dataset usability: False
Dataset is not usable for trait-gene association studies due to bias or other issues.
